# Notebook 3: Symbolic forms

Underworld is deeply integrated with `sympy` [(www.sympy.org)](www.sympy.org) so that any mesh variable can also be used in a sympy expression. We already saw `sympy` expressions for the coordinates and coordinate directions.

In the examples below, we use a simple 2D, Cartesian mesh because it is much simpler to see the various changes.

In [25]:
#|  echo: false 
# This is required to fix pyvista 
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import underworld3 as uw
import numpy as np
import sympy

In [3]:
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords = (-1.0, -1.0),
    maxCoords = (+1.0, +1.0),
    cellSize = 0.05,
    regular=True,
    verbose=False,
)

x,y = mesh.CoordinateSystem.X

As before, we add discrete variables

In [4]:
# mesh variable example / test

s1 = uw.discretisation.MeshVariable(
    varname="S1",
    mesh=mesh, 
    vtype = uw.VarType.SCALAR,
    varsymbol=r"s_{[1]}"
)

s2 = uw.discretisation.MeshVariable(
    varname="S2",
    mesh=mesh, 
    vtype = uw.VarType.SCALAR,
    varsymbol=r"s_{[2]}"
)

v = uw.discretisation.MeshVariable(
    varname="V1",
    mesh=mesh, 
    degree=2,
    vtype = uw.VarType.VECTOR,
    varsymbol=r"\mathbf{v}",
)



## Symbolic forms, derivatives

Variables can be part of complicated `sympy` expressions. It is important to note that all symbols are matrices and `sympy` can be fussy when it comes to operations with other matrices (scalars are not entirely equivalent to $1 \times 1$ matrices).

In [5]:
s1.sym[0]+s2.sym[0] + v.sym[0]

{\mathbf{v}}_{ 0 }(N.x, N.y) + {s_{[1]}}(N.x, N.y) + {s_{[2]}}(N.x, N.y)

Derivatives can be handled explicitly, but the mesh also provides vector operators and these are generally better because they are automatically consistent with the underlying coordinate system for the mesh. 

For compound expressions of variables, use `mesh.vector.curl(expression)` but for individual variables, `variable.curl()` is an equivalent shorthand.

In [6]:
# grad by hand
s1.sym[0].diff(x) + s1.sym[0].diff(y)

{s_{[1]}}_{,0}(N.x, N.y) + {s_{[1]}}_{,1}(N.x, N.y)

In [7]:
# grad
s1.gradient()

Matrix([[{s_{[1]}}_{,0}(N.x, N.y), {s_{[1]}}_{,1}(N.x, N.y)]])

In [8]:
v.curl()

-{\mathbf{v}}_{ 0,1}(N.x, N.y) + {\mathbf{v}}_{ 1,0}(N.x, N.y)

In [9]:
# curl
mesh.vector.curl(s1.sym * v.sym)

-{\mathbf{v}}_{ 0 }(N.x, N.y)*{s_{[1]}}_{,1}(N.x, N.y) - {\mathbf{v}}_{ 0,1}(N.x, N.y)*{s_{[1]}}(N.x, N.y) + {\mathbf{v}}_{ 1 }(N.x, N.y)*{s_{[1]}}_{,0}(N.x, N.y) + {\mathbf{v}}_{ 1,0}(N.x, N.y)*{s_{[1]}}(N.x, N.y)

In [10]:
# v dot grad (scalar)... 
v.sym.dot(mesh.vector.gradient(s1.sym))

{\mathbf{v}}_{ 0 }(N.x, N.y)*{s_{[1]}}_{,0}(N.x, N.y) + {\mathbf{v}}_{ 1 }(N.x, N.y)*{s_{[1]}}_{,1}(N.x, N.y)

Symbolic forms can be evaluated at points in the (meshed domain) using `uw.function.evaluate`. Pure sympy functions can be used to set values in the data container of a `meshVariable` object 

In [17]:
with mesh.access(s1, s2):
    s1.data[:,0] = uw.function.evaluate(sympy.cos(3 * sympy.pi * x)**2 , s1.coords)
    s2.data[:,0] = uw.function.evaluate(sympy.sin(3 * sympy.pi * y)**2 , s2.coords)


In [26]:
s1.view()

**Class**: <class 'underworld3.discretisation._MeshVariable'>

**MeshVariable:**

  > symbol:  ${s_{[1]}}$

  > shape:   $(1, 1)$

  > degree:  $1$

  > continuous:  `True`

  > type:    `SCALAR`

**FE Data:**


  > PETSc field id:  $0$ 

  > PETSc field name:   `S1` 

array([[1.        ],
       [1.        ],
       [1.        ],
       ...,
       [0.79389263],
       [0.79389263],
       [0.79389263]])

In [98]:
# Visualise it / them

import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["s1"] = vis.scalar_fn_to_pv_points(pvmesh, s1.sym[0])
pvmesh.point_data["s2"] = vis.scalar_fn_to_pv_points(pvmesh, s2.sym[0])
pvmesh.point_data["s1s2"] = vis.scalar_fn_to_pv_points(pvmesh, s1.sym[0]*s2.sym[0])

pvmesh.warp_by_scalar(scalars="s1s2", factor=0.3, normal=(0,0,1), inplace=True)

# pvmesh.plot(show_edges=True, show_scalar_bar=False)

pl = pv.Plotter(window_size=(750, 750))

pl.add_mesh(pvmesh, 
            show_edges=True,
            edge_color="#4455FF",
            cmap="Greys",
            scalars="s1s2", 
            show_scalar_bar=False)

# Save and show the mesh

pl.camera_position = 'yz'
pl.camera.azimuth = 45
pl.camera.elevation = 45

pl.export_html("html5/sine_squared.html")

In [99]:
#| fig-cap: "Interactive Image: Square mesh of triangular elements on which we evaluated a simple `sympy` function of position"

from IPython.display import IFrame
IFrame(src="html5/sine_squared.html", width=600, height=400)


## More information

sympy documentation ... 

More examples ... 

